In [2]:
import pandas as pd
import datetime

In [16]:
ibge = pd.read_csv('./dados/ibge_drs.csv')

#função para iterar datas
def daterange(start_date, end_date):
    for i in range(int((end_date - start_date).days)):
        yield start_date + datetime.timedelta(i)

inicio = datetime.datetime(2020,1,1)
fim = datetime.datetime.now()

for cod in ibge.iloc():
    cidade = str(cod['Código IBGE'])
    co = pd.read_csv('casos_obitos/' + cidade + '.csv')
    vacinacao = pd.read_csv('vacinacao/' + cidade + '.csv')
    #isolamento = pd.read_csv('isolamento/cidades/' + cidade + '.csv')
    ocupacao = pd.read_csv('ocupacao/' + cidade + '.csv')
    #r = pd.read_csv('r_efetivo/cidades/' + cidade + '.csv')
    print(ocupacao.columns)
    dados = { 'Município': [], 'Código IBGE': [], 'Populacao': [], 'Total 1ª Dose': [],
       'Total 2ª Dose': [], 'Total Unica': [], 'Total Doses Aplicadas': [],
       'População Vacinada dose1/População Total': [],
       'População Vacinada/População Total': [], 'data': [], 'diagnostico_covid19': [], 'obito': [], 'media_movel_casos': [],
       'media_movel_obitos': [], 'ocupacao_leitos': []}
    novo = pd.DataFrame(data=dados)
    for data in daterange(inicio, fim):
        vac = vacinacao[vacinacao['data'] == data.strftime('%Y-%m-%d')].drop(columns=['Unnamed: 0'])
        cao = co[co['data_inicio_sintomas'] == data.strftime('%Y-%m-%dT00:00:00Z')].drop(columns=['Unnamed: 0','nome_munic', 'codigo_ibge', 'data_inicio_sintomas'])
        cao['data'] = data.strftime('%Y-%m-%d')
        ocu = ocupacao[ocupacao['data'] == data.strftime('%Y-%m-%d')].drop(columns=['Unnamed: 0', 'cod_ibge', 'data'])
        ocu['data'] = data.strftime('%Y-%m-%d')
        soma = pd.merge(vac, cao, on="data")
        soma = pd.merge(soma, ocu, on="data")
        
        if(not(vac.empty) and not(cao.empty) and not(ocu.empty)):
            novo = novo.append(soma)
    novo = novo.reset_index().drop(columns='index')
    break

#print(novo.dropna())
novo

Index(['Unnamed: 0', 'cod_ibge', 'data', 'ocupacao_leitos'], dtype='object')


,Município,Código IBGE,Populacao,Total 1ª Dose,Total 2ª Dose,Total Unica,Total Doses Aplicadas,População Vacinada dose1/População Total,População Vacinada/População Total,data,diagnostico_covid19,obito,media_movel_casos,media_movel_obitos,ocupacao_leitos
0,ADAMANTINA,3500105.0,35111.0,2068.0,543.0,0.0,2611.0,5.889892,1.546524,2021-02-21,11.0,1.0,16.571429,0.571429,"79,81"
1,ADAMANTINA,3500105.0,35111.0,2079.0,547.0,0.0,2626.0,5.921221,1.557916,2021-02-22,24.0,1.0,17.428571,0.571429,"80,29"
2,ADAMANTINA,3500105.0,35111.0,2104.0,608.0,0.0,2712.0,5.992424,1.731651,2021-02-23,13.0,0.0,15.142857,0.571429,"80,77"
3,ADAMANTINA,3500105.0,35111.0,2110.0,615.0,0.0,2725.0,6.009513,1.751588,2021-02-24,10.0,1.0,14.285714,0.571429,"79,81"
4,ADAMANTINA,3500105.0,35111.0,2125.0,739.0,0.0,2864.0,6.052234,2.104753,2021-02-25,7.0,1.0,11.714286,0.714286,"77,03"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,ADAMANTINA,3500105.0,35111.0,21559.0,7968.0,632.0,30159.0,61.402410,24.493748,2021-07-16,6.0,0.0,9.857143,0.142857,"72,95"
146,ADAMANTINA,3500105.0,35111.0,21747.0,7997.0,632.0,30376.0,61.937854,24.576344,2021-07-17,3.0,0.0,8.142857,0.000000,"68,59"
147,ADAMANTINA,3500105.0,35111.0,21747.0,7997.0,632.0,30376.0,61.937854,24.576344,2021-07-18,5.0,0.0,7.714286,0.000000,"65,58"
148,ADAMANTINA,3500105.0,35111.0,21747.0,7997.0,632.0,30376.0,61.937854,24.576344,2021-07-19,6.0,0.0,7.285714,0.000000,"67,75"


In [6]:
data = {'Name': ['Tom', 'Joseph', 'Krish', 'John'], 'Age': [20, 21, 19, 18]} 
data2 = {'Name': ['Tom', 'Joseph', 'Krish', 'John'], 'alt': [21, 21, 19, 18]} 
  
# Create DataFrame  
df = pd.DataFrame(data) 
df2 = pd.DataFrame(data2) 
  
# Print the output.  
print(df)  
print(df2)  

     Name  Age
0     Tom   20
1  Joseph   21
2   Krish   19
3    John   18
     Name  alt
0     Tom   21
1  Joseph   21
2   Krish   19
3    John   18


In [11]:
result = pd.merge(df, df2, on="Name")

TypeError: merge() missing 1 required positional argument: 'right'

In [10]:
result

,Name,Age,alt
0,Tom,20,21
1,Joseph,21,21
2,Krish,19,19
3,John,18,18


In [13]:
ibge = pd.read_csv('./dados/ibge_drs.csv')

#função para iterar datas
def daterange(start_date, end_date):
    for i in range(int((end_date - start_date).days)):
        yield start_date + datetime.timedelta(i)

inicio = datetime.datetime(2020,1,1)
fim = datetime.datetime.now()

for cod in ibge.iloc():
    cidade = str(cod['Código IBGE'])
    co = pd.read_csv('casos_obitos/' + cidade + '.csv')
    vacinacao = pd.read_csv('vacinacao/' + cidade + '.csv')
    #isolamento = pd.read_csv('isolamento/cidades/' + cidade + '.csv')
    ocupacao = pd.read_csv('ocupacao/' + cidade + '.csv')
    #r = pd.read_csv('r_efetivo/cidades/' + cidade + '.csv')
    print(co.columns)
    dados = {'nome_munic': [], 'codigo_ibge': [], 'data': [], 'diagnostico_covid19': [], 'obito': [], 'media_movel_casos': [],
       'media_movel_obitos': [], 'Populacao': [], 'Total 1ª Dose': [], 'Total 2ª Dose': [],
       'Total Unica': [], 'Total Doses Aplicadas': [], 'População Vacinada dose1/População Total': [],
       'População Vacinada/População Total': [], 'ocupacao_leitos': []}
    novo = pd.DataFrame(data=dados)
    for data in daterange(inicio, fim):
        soma = vacinacao[vacinacao['data'] == data.strftime('%Y-%m-%d')].drop(columns=['Unnamed: 0'])
        soma[['diagnostico_covid19', 'obito', 'media_movel_casos','media_movel_obitos']] = co[co['data_inicio_sintomas'] == data.strftime('%Y-%m-%dT00:00:00Z')].drop(columns=['Unnamed: 0','nome_munic', 'codigo_ibge', 'data_inicio_sintomas'])
        soma['ocupacao_leitos'] = ocupacao[ocupacao['data'] == data.strftime('%Y-%m-%d')].drop(columns=['Unnamed: 0', 'cod_ibge', 'data'])
        #print(soma)
       
        novo = novo.append(soma, ignore_index=True)
            
    break

#print(novo.dropna())
novo

Index(['Unnamed: 0', 'nome_munic', 'codigo_ibge', 'data_inicio_sintomas',
       'diagnostico_covid19', 'obito', 'media_movel_casos',
       'media_movel_obitos'],
      dtype='object')


,nome_munic,codigo_ibge,data,diagnostico_covid19,obito,media_movel_casos,media_movel_obitos,Populacao,Total 1ª Dose,Total 2ª Dose,Total Unica,Total Doses Aplicadas,População Vacinada dose1/População Total,População Vacinada/População Total,ocupacao_leitos,Município,Código IBGE
0,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,NaN,NaN,2021-07-22,NaN,NaN,NaN,NaN,35111.0,22903.0,9190.0,879.0,32972.0,65.230270,28.677622,NaN,ADAMANTINA,3500105.0
495,NaN,NaN,2021-07-23,NaN,NaN,NaN,NaN,35111.0,23118.0,9607.0,879.0,33604.0,65.842613,29.865284,NaN,ADAMANTINA,3500105.0
496,NaN,NaN,2021-07-24,NaN,NaN,NaN,NaN,35111.0,23252.0,9665.0,879.0,33796.0,66.224260,30.030475,NaN,ADAMANTINA,3500105.0
497,NaN,NaN,2021-07-25,NaN,NaN,NaN,NaN,35111.0,23252.0,9665.0,879.0,33796.0,66.224260,30.030475,NaN,ADAMANTINA,3500105.0
